In [89]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from sqlalchemy import create_engine
from config import db_password


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
#  Import and read the charity_data.csv.
import pandas as pd 
postgresql://[user]:[password]@[location]:[port]/[database]
divorce_df = pd.read_csv("Data/analytic_data2010.csv")
divorce_df.head()

In [100]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

clean_divorce_df = divorce_df.drop(["fips_code", "Year"], axis=1)
clean_divorce_df.head()


MemoryError: Unable to allocate 5.01 GiB for an array with shape (34299, 19611) and data type float64

In [ ]:
# Split our preprocessed data into our features and target arrays
y = clean_divorce_df["Div_Rate"].values
X = clean_divorce_df.drop(["Div_Rate"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\wobbi\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 156920    
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 156,971
Trainable params: 156,971
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 37934.1133 - accuracy: 0.5000
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 27967.5723 - accuracy: 0.5012
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 30008.2402 - accuracy: 0.5055
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 28666.6230 - accuracy: 0.5019
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 20232.5898 - accuracy: 0.5063
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 25769.9043 - accuracy: 0.4995
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 14119.4141 - accuracy: 0.4992
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 2395.6096 - accuracy: 0.4965A: 0s - loss: 2308.6321 - accura
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 1559.8740 - accuracy: 0.5047
Epoch 10/100
804/804 [===

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5776 - accuracy: 0.7009 - 409ms/epoch - 2ms/step
Loss: 0.577617883682251, Accuracy: 0.7008746266365051
